# Moist air properties

Let's consider a volume flow rate of 10 m³/s of humid air at 25 °C, relative humidity 80 % in an HVAC system located at 1000 m of altitude.

In these conditions, find:
- the partial pressure of the dry air;
- the water vapor content (or the humidity ratio);
- the density of the humid air;
- the specific volume of the humid air;
- the specific enthalpy of the humid air;
- the mass flow rate of humid air;
- the mass flow rate of dry air;
- the mass flow rate of water vapor that needs to be added to the air flow in order to obtain saturated air.

In [1]:
import numpy as np

# Data
V = 10   # m³/s, volume flow rate
θ = 25   # °C, air temperature
ϕ = 0.8  # -, relative humidity
z = 1000 # m, altitude

T = θ + 273.15 # K, air temperature

**Note**: [Unicode symbols](https://www.compart.com/en/unicode), such as the Greek ketters θ or ϕ, [subscripts and superscripts](https://en.wikipedia.org/wiki/Unicode_subscripts_and_superscripts#Superscripts_and_subscripts_block) can be used. 

In [2]:
# Physical constants
l = 2495e3  # J/kg, specific latent heat for vaporization
c = 1e3     # J/(kg·K), specific heat of dry air
cv = 1.96e3 # J/(kg·K), specific heat of water vapor

M = 28.9645 # kg/kmol, molar mass of dry air
Mv = 18.015 # kg/kmol, molar mass of water vapor
R = 8314    # J/(kmol·K), gas constant

## Partial pressure of dry air

The partial pressure of the dry air, $p_{da}$, is:

$$ p_{da} = p - p_v = p - \phi p_{vs}$$

where:
- $p$ is the total pressure, Pa;
- $p_{v}$ - [vapour pressure of water](https://en.wikipedia.org/wiki/Vapour_pressure_of_water), Pa;
- $p_{vs}$ - [saturation vapour pressure](https://en.wikipedia.org/wiki/Vapour_pressure_of_water), Pa;
- $\phi$ - [relative humidity](https://en.wikipedia.org/wiki/Humidity) of the air-water mixture, dimensionless.


In [3]:
# air pressure as a function of altitude
p = 101.325e3 * (1 - 2.25577e-5 * z)**5.2559   # Pa

In [4]:
def pvs(θ: float) -> float:
    """
    Saturation vapor pressure over liquid water, in Pa,
    as a function of tempetature θ, in °C
    Valid for temperaure range 0 ... 200°C
    ASHRAE-01 (2017) eq. (6), p. 1.8
    """
    import numpy as np
    T = θ + 273.15      # K, temperature

    C8 = -5.800_220_6e3
    C9 = 1.391_499_3e0
    C10 = -4.864_023_9e-2
    C11 = 4.176_476_8e-5
    C12 = -1.445_209_3e-8
    C13 = 6.545_967_3e0
    return np.exp(C8/T + C9 + C10 * T + C11 * T **2 + C12 * T **3 + C13 * np.log(T))  # Pa

In [5]:
# vapor pressure as a function of relative humidity ϕ and temperature θ
pv = ϕ * pvs(θ)    # Pa

In [6]:
# dry-air partial pressure
pda = p - pv       # Pa
print(f"Partial pressure of the dry air is: {pda:.0f} Pa = {pda/1000:.1f} kPa")

Partial pressure of the dry air is: 87339 Pa = 87.3 kPa


## Humidity ratio (mixing ratio)

Water content (or humidity ratio) is (ASHRAE-F01 (2017), eq. 20, p. 1.9):

$$w ≡ \frac{m_v}{m} = \frac{\dot{m_v}}{\dot{m}} = \frac{M_v}{M}\frac{p_v}{p -p_v}$$

where:
- $\dot{m}_v$ is the mass flow rate of vapor, kg/s;
- $\dot{m}$ - mass flow rate of dry air, kg/s;
- $M = 28.9645 \, \mathrm{kg/kmol}$ - molar mass of dry air;
- $M_v = 18.015 \, \mathrm{kg/kmol}$ - molar mass of water vapor;
- $p$ - total pressure of humid air, Pa;
- $p_v$ - vapor pressure, Pa.

*Note* (ASHRAE-F01 (2017), p. 1.8):
- *humidity ratio*, $w$, is the mass of water vapor divided by the mass of dry air, $w = m_v /m$;
- *specific humidity*, $\gamma$, is the mass of water vapor divided by the mass of moist air, $\gamma = m_v /(m_v + m)$.

In [7]:
w = Mv / M * pv / (p - pv)  # kg/kg, humidity ratio (mass vapor / mass dry air)
print(f"Water content is: {1000 * w:.2f} g/kg (mass of vapor per mass of dry air)")

Water content is: 18.06 g/kg (mass of vapor per mass of dry air)


## Density of moist air

The density of humid air is 
$$\rho = \frac{m_{ha}}{V} = \frac{M}{R} \frac{p}{T}+\frac{M_v}{R}\frac{p_v}{T}$$


In [8]:
ρ = M / R * pda / T + Mv / R * pv / T   # kg/m³, density
print(f'Density of moist air is: {ρ:.2f} kg/m³')

Density of moist air is: 1.04 kg/m³


Equivalently,

$$\rho = \frac{M}{R} \frac{p}{T} - \phi \frac{M - M_v}{R} \frac{p_{vs}}{T}$$

This last formulation emphasizes that the density of the moist air, $\rho$, decreases with the increase of the  relative humidity, $\phi$.

In [9]:
ρ = M / R * p / T  - ϕ * (M - Mv) / R * pvs(θ) / T
print(f'For ϕ = {100 * ϕ:.0f} %, density of moist air is: {ρ:.2f} kg/m³')

ρda = M / R * p / T
print(f'For ϕ = 0 %, density of moist air is: {ρda:.2f} kg/m³')

For ϕ = 80 %, density of moist air is: 1.04 kg/m³
For ϕ = 0 %, density of moist air is: 1.05 kg/m³


The relative difference between the density of the humid air $(\theta = 0°C, \phi = 80 \, \%)$ and the dry air $(\theta = 0°C, \phi = 0 \, \%)$ is relatively small, but as a result of this difference, the humid air is above in a stratified atmosphere.

In [10]:
print(f'Relative difference between densities of \
humid and dry air is: {(ρ - ρda) / ρ * 100:.1f} %')

Relative difference between densities of humid and dry air is: -1.1 %


## Specific volume

The specific volume is:
$$v = \frac{V}{m} = \frac{1}{\rho} = \frac{R}{M_v} \left (\frac{M_v}{M} + w  \right ) \frac{T}{p}$$

In [11]:
v = R / Mv * (Mv / M + w) * T / p
print(f'Specific volume is: {v:.2f} m³/kg')

Specific volume is: 0.98 m³/kg


In the field of air conditioning, **specific** means relative to the mass of dry air, $m$, while **massic** means relative to the mass of humid air $m_{ha}$, which is the mass dry air, $m$, plus the mass of vapor, $m_v$:

$$m_{ha} = m + m_v$$

The specific volume coresponds to the volume divided by the mass of dry air, $m$. Therefore $v \neq \frac{1}{\rho}$: the specific volume, $v = \frac{V}{m}$, is different of the massic volume,  $\frac{1}{\rho} = \frac{V}{m_{ha}}$, where $m$ is the mass of dry air and $m_{ha}$ is the mass of humid air.

In [12]:
print(f'Massic volume 1/ρ is: {1 / ρ:.2f} m³/kg, different of')
print(f'specific volume, v : {v:.2f} m³/kg')

Massic volume 1/ρ is: 0.96 m³/kg, different of
specific volume, v : 0.98 m³/kg


## Specific enthalpy

Specific ethalpy is the enthalpy, $H$, for the unit of mass of dry air,

$$h = \frac{H}{m} = c\theta + w(l + c_v \theta)$$

In [13]:
h = c * θ + w * (l + cv * θ)
print(f"Specific enthalpy is: {h/1000:.3f} kJ/kg")

Specific enthalpy is: 70.932 kJ/kg


If we neglect the sensible enthalpy of the water vapor, $c_v \theta$, then the specific enthalpy is (Ghiaus 2014):
$$h = c \theta + w l$$

In [14]:
h1 = c * θ + w * l
print(f"Specific enthalpy with simplified formula is: {h1/1000:.3f} kJ/kg")

Specific enthalpy with simplified formula is: 70.048 kJ/kg


The relative error due to this approximtion is smaller than 5 % (Ghiaus 2014).

In [15]:
print(f"Relative error done by neglecting the sensible heat of water vapor:\
{(h1 - h) / h * 100:.3f} %")

Relative error done by neglecting the sensible heat of water vapor:-1.247 %


## Mass flow rate of humid air

The mass flow rate of humid air, i.e. dry air and vapor, is:
$$\dot m_{ha} = \frac{\rho}{\dot{V}}$$

In [16]:
mha = ρ * V     # kg/s mass flow rate of humid air
print(f'Mass flow rate of humid air is: {mha:.1f} kg/s')

Mass flow rate of humid air is: 10.4 kg/s


## Mass flow rate of dry air

The mass flow rate of dry air is:

$$\dot{m} = \frac{\dot{V}}{v}$$

where:
- $\dot{V}$ is the volumetric flow rate of humid air, m³/s;
- $v$ - specific volume of moist air, m³/kg.

In [17]:
m = V / v     # kg/s, mass flow rate of humid air
print(f"Mass flow rate of dry air: {m:.1f} kg/s")

Mass flow rate of dry air: 10.2 kg/s


*Note*: The mass flow rate of dry air, $\dot{m}$, is used in psychrometric analysis of air conditioning systems. In order to find the volumetric flow rate, $\dot{V}$, which is used in aeraulic calculations (i.e. to size the air ducts and the fans), the relation:

$$\dot{V} = v \dot{m}$$

is used, where:
- $v$ - specific volume of moist air, m³/kg;
- $\dot{m}$ - mass flow rate of dry air, kg/s.

## Mass flow rate of water for saturation

The humidity ratio at saturation is

$$w_s = \frac{\dot{m_v}}{\dot{m}} = \frac{M_v}{M}\frac{p_{vs}}{p -p_{vs}}$$

where:
- $\dot{m}_v$ is the mass flow rate of vapor, kg/s;
- $\dot{m}$ - mass flow rate of dry air, kg/s;
- $M = 28.9645 \, \mathrm{kg/kmol}$ - molar mass of dry air;
- $M_v = 18.015 \, \mathrm{kg/kmol}$ - molar mass of water vapor;
- $p$ - total pressure of humid air, Pa;
- $p_v$ - vapor pressure, Pa.

In [18]:
ws = Mv / M * pvs(θ) / (p - pvs(θ))    # kg/kg, water content at saturation

The mass flow-rate of vapor which is needed to be added to the mass flow-rate of humid air in order to saturate the air is:

$$\dot{m_v} = (w_s - w) \dot{m}$$

where:
- $w_s$ is water content (or humidity ratio) at saturation;
- $w$ - water content (or humidity ratio) of the moist air;
- $\dot{m}$ - mass flow rate of dry air, kg/s.

In [19]:
mv = (ws - w) * m                      # kg/s, mass flow of water needed to saturate
print(f'Mass flow rate of water needed to saturate the air flow is: \
{1000 * mv:.3f} g/s or {3600 * mv:.0f} kg/h.')

Mass flow rate of water needed to saturate the air flow is: 47.749 g/s or 172 kg/h.


## Bibliography

1. ASHRAE (2009) Fundamentals, ASHRAE Handbook (2009). Chapter 1: Psychrometrics. *American Society of Heating, Refrigerating and Air-Conditioning Engineers*. Atlanta, USA

2. ASHRAE-F01 (2017) F01 Psychrometrics, in [ASHRAE Fundamentals (SI Edition)](https://archive.org/details/ashraehandbook2017hvacfundamentalssi/mode/1up), *American Society of Heating, Refrigerating and Air-Conditioning Engineers*. Atlanta, USA

3. TRANE (2012) [Psychrometry: One of the Fundamental Series](https://www.tranebelgium.com/files/book-doc/19/en/19.vwljh8zn.pdf)

4. Ghiaus, C. (2014). Linear algebra solution to psychometric analysis of air-conditioning systems. *Energy, 74*, 555-566. DOI: [10.1016/j.energy.2014.07.021](https://doi.org/10.1016/j.energy.2014.07.021)